# Test my Neural Nets classes

Should do [Working efficiently with jupyter lab](https://florianwilhelm.info/2018/11/working_efficiently_with_jupyter_lab/)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
#%matplotlib inline

In [2]:
# import Importing_Notebooks
import numpy as np
#from scipy import ndimage
import matplotlib.pyplot as plt
import dill

In [3]:
from nn import Network, Layer, IdentityLayer, AffinityLayer, MapLayer
from nnbench import NNBench

___

## Tests

### One identity layer
See if the wheels turn:

In [4]:
net = Network()
net.extend(IdentityLayer())
all(net(np.arange(3)) == np.arange(3))

True

It does not learn, as expected:

In [5]:
facts = [(np.arange(2*n, 2*n+2), np.arange(2*n+1, 2*n-1, -1)) for n in range(3)]
net.learn(facts)

1.0

In [6]:
net(np.arange(2,4))

array([2, 3])

### One map layer

In [7]:
net = Network()
net.extend(MapLayer(lambda x: x+1, lambda d: 1))
all(net(np.arange(3)) == np.arange(3)+1)

True

It does not learn, as expected:

In [8]:
net.learn(facts), all(net(np.arange(5)) == np.arange(5)+1), net(np.arange(2,4))

(2.0, True, array([3, 4]))

### One affine layer

In [9]:
net = Network()
net.extend(AffinityLayer(2,2))

In [10]:
t = net.layers[0]
t.M, t.b

(array([[ 0.99766408,  0.56255976],
        [ 1.02137928, -0.1366786 ]]),
 array([-2.26151733, -0.15278411]))

#### Can it learn the identity transformation?

In [11]:
bench = NNBench(net)
bench.checkpoint_net()
bench.learning_potential()

19.402077727135147

In [12]:
bench.plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Exchange transform

In [13]:
bench.ideal = lambda v: np.array([v[1], v[0]])
bench.net.layers[0].randomize()
bench.knobs_plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Learn thru a map layer

This layer squares its input and divides by two:

In [14]:
net = Network()
net.extend(AffinityLayer(2,2))

def dtanh(x):
    v = np.tanh(x)
    return (1+v)*(1-v)

net.extend(MapLayer(lambda x:x*x/2.0, lambda d:d))
#net.extend(MapLayer(np.tanh, dtanh))
bench = NNBench(net)
bench.checkpoint_net()

In [15]:
net.layers[0].M, net.layers[0].b

(array([[-0.32158469,  0.15113037],
        [-0.01862772,  0.48352879]]),
 array([0.76896516, 1.36624284]))

#### Can it learn difference squared?

In [16]:
bench.ideal = lambda v: [(v[0]-v[1])**2,0]
#bench.ideal = lambda v: [(v[0]>0)*2-1,(v[0]>v[1])*2-1]
bench.learning_potential()
#bench.knobs_plot_learning(100)

/home/jovyan/work/nbs/nn.py:174: RuntimeWarning: overflow encountered in multiply
  input_delE = self.vdfundx(self.input) * output_delE
/home/jovyan/work/nbs/nn.py:152: RuntimeWarning: invalid value encountered in subtract
  self.M -= np.einsum('i,j', output_delE, self.input) # use np.outer?
/home/jovyan/work/nbs/nn.py:153: RuntimeWarning: invalid value encountered in subtract
  self.b -= output_delE


nan

In [17]:
bench.plot_learning(100)

In [18]:
bench.knobs_plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### add a RELU

In [19]:
bench2 = NNBench(Network())
bench2.net.layers = []
bench2.net.extend(AffinityLayer(2,2))
leak = 0
bench2.net.extend(MapLayer(lambda x: (x*(1+leak/2)+abs(x)*(1-leak/2))/2, lambda d: [leak,1][1 if d>0 else 0]))
bench2.net.layers

[<nn.AffinityLayer at 0x7fa7f6e0d150>, <nn.MapLayer at 0x7fa7f6e0d190>]

In [19]:
bench2.ideal = lambda v: [(v[0]-v[1])**2,0]
bench2.learning_potential()

0.5069783889936366

In [20]:
bench2.knobs_plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
bench3 = NNBench(net)
net.layers = []
net.extend(AffinityLayer(2,4))
net.extend(AffinityLayer(4,2))
[l.randomize() for l in net.layers]
bench3.checkpoint_net()
bench3.ideal = lambda v: [(v[0]-v[1])**2,0]
bench3.knobs_plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jovyan/work/nbs/nn.py:151: RuntimeWarning: invalid value encountered in matmul
  input_delE = self.M.T @ output_delE
/home/jovyan/work/nbs/nn.py:147: RuntimeWarning: invalid value encountered in add
  self.output = self.M @ x + self.b


### XOR

In [22]:
net = Network()
net.extend(AffinityLayer(2,2))

In [23]:
t = net.layers[0]
t.M, t.b

(array([[-0.32158469,  0.15113037],
        [-0.01862772,  0.48352879]]),
 array([0.76896516, 1.36624284]))